# Importacion de librerias

In [1]:
# Se importan las bibliotecas
from __future__ import absolute_import, division, print_function, unicode_literals
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="-1";

import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
!wget -q https://raw.githubusercontent.com/JefeLitman/VideoDataGenerator/master/DatasetsLoader.py -O DatasetsLoader.py
from DatasetsLoader import VideoDataGenerator

# Creacion del tf_record

In [2]:
def _bytes_feature(value):  #Lo utilizamos para los datos ya como tal (imagenes o volumenes)
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def video_example(video, label):
    feature = {
        'frames': _int64_feature(video.shape[0]),
        'height': _int64_feature(video.shape[1]),
        'width': _int64_feature(video.shape[2]),
        'depth': _int64_feature(video.shape[3]),
        'label': _int64_feature(label),
        'video_raw': _bytes_feature(video.tostring()),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [3]:
root_path = "/home/jefelitman/DataSets/hmdb51/split_1"

batch_size = 1
original_size = [89,67]
size = [58,58]
frames = 60
canales = 3

def custom_steps_temporal(frames):
    paso = len(frames)//60
    videos = []
    for i in range(paso):
        indices = range(i, len(frames),paso)
        videos.append([frames[j] for j in indices[:60]])
    return videos
def flip_vertical(volume):
    return np.flip(volume, (0, 2))[::-1]
def video_transf(video):
    escalador = MinMaxScaler()
    new_video = video.reshape((video.shape[0]*video.shape[1]*video.shape[2]*video.shape[3],1))
    new_video = escalador.fit_transform(new_video)
    return new_video.reshape((video.shape[0],video.shape[1],video.shape[2],video.shape[3]))

dataset = VideoDataGenerator(directory_path = root_path, 
                             table_paths = None, 
                             batch_size = batch_size, 
                             original_frame_size = original_size, 
                             frame_size=size, 
                             video_frames = frames, 
                             temporal_crop = ("custom", custom_steps_temporal),
                             video_transformation = [("augmented",flip_vertical),("full", video_transf)],
                             frame_crop = (None, None), 
                             shuffle = False,
                             shuffle_after_epoch = False,
                             conserve_original = False)

In [9]:
save_path = '/home/jefelitman/DataSets/hmdb51'

record = os.path.join(save_path,'hmdb_split_1_train.tfrecord')
with tf.io.TFRecordWriter(record) as writer:
    for b in range(dataset.train_batches):
        video , label = dataset.get_next_train_batch(1)
        tf_example = video_example(video[0], label[0])
        writer.write(tf_example.SerializeToString())
        
record = os.path.join(save_path,'hmdb_split_1_test.tfrecord')
with tf.io.TFRecordWriter(record) as writer:
    for b in range(dataset.test_batches):
        video , label = dataset.get_next_test_batch(1)
        tf_example = video_example(video[0], label[0])
        writer.write(tf_example.SerializeToString())